# Bond Option Pricing using Various Models

Value an option on a coupon paying bond using the Hull-White, Black-Derman-Toy and Black-Karasinski model

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [32]:
from financepy.utils import *
from financepy.market.curves import *
from financepy.models.bk_tree import BKTree
from financepy.models.bdt_tree import BDTTree
from financepy.models.hw_tree import HWTree
from financepy.products.bonds import *

## Set up the Discount Curve

In [33]:
settle_date = Date(1, 12, 2019)

Set up discount curve

In [34]:
rate = 0.10
dcType = DayCountTypes.THIRTY_360_BOND
fixedFreq = FrequencyTypes.SEMI_ANNUAL
discount_curve = DiscountCurveFlat(settle_date, rate, fixedFreq, dcType)

## Set up the Bond Option

First create the bond

In [35]:
issue_date = Date(1,12,2018)
maturity_date = issue_date.add_tenor("3Y")
coupon = 0.10
frequencyType = FrequencyTypes.SEMI_ANNUAL
dc_type = DayCountTypes.THIRTY_360_BOND
bond = Bond(issue_date, maturity_date, coupon, frequencyType, dc_type)

Let's first price the bond on the libor curve

In [36]:
cp = bond.clean_price_from_discount_curve(settle_date, discount_curve)
dp = bond.dirty_price_from_discount_curve(settle_date, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   100.000
Fixed Income Dirty Price:   100.000


In [37]:
settle_date

01-DEC-2019

In [38]:
bond.print_payments(settle_date)

 01-JUN-2020      5.00000 
 01-DEC-2020      5.00000 
 01-JUN-2021      5.00000 
 01-DEC-2021    105.00000 



# Create the Bond Options

Then define and create the option

In [39]:
expiry_date = settle_date.add_tenor("18m")
strike_price = 95.0
face_amount = 100.0

In [52]:
europeanCall = BondOption(bond, expiry_date, strike_price, OptionTypes.EUROPEAN_CALL)
americanCall = BondOption(bond, expiry_date, strike_price, OptionTypes.AMERICAN_CALL)

In [53]:
europeanPut = BondOption(bond, expiry_date, strike_price, OptionTypes.EUROPEAN_PUT)
americanPut = BondOption(bond, expiry_date, strike_price, OptionTypes.AMERICAN_PUT)

Consider the forward bond price.

In [56]:
cp = bond.clean_price_from_discount_curve(expiry_date, discount_curve)
dp = bond.dirty_price_from_discount_curve(expiry_date, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   100.000
Fixed Income Dirty Price:   100.000


In [57]:
df = discount_curve.df(expiry_date)
callIntrinsic = max(cp - strike_price, 0) * df
putIntrinsic = max(strike_price - cp, 0) * df

In [58]:
callIntrinsic

4.319187992657368

In [59]:
putIntrinsic

0.0

## Tree Models

Set the model parameters, start with zero vol.

In [60]:
num_steps = 200

In [61]:
sigma = 0.00001
a = 0.1
modelHW = HWTree(sigma*rate, a, num_steps)
modelBK = BKTree(sigma, a, num_steps)
modelBDT = BDTTree(sigma, num_steps)

Check some limits - the call option should be worth the intrinsic.

In [62]:
ec = europeanCallBondOption.value(settle_date, discount_curve, modelHW)
ac = americanCallBondOption.value(settle_date, discount_curve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   4.30649
American Call Value:   4.96842


The put option too.

In [63]:
ep = europeanPutBondOption.value(settle_date, discount_curve, modelHW)
ap = americanPutBondOption.value(settle_date, discount_curve, modelHW)
print("European Put Value: %9.5f" % ep)
print("American Put Value: %9.5f" % ap)

European Put Value:   0.00000
American Put Value:   0.00000


Now we turn on the volatility.

In [64]:
sigma = 0.20
a = 0.1
modelHW = HWTree(sigma*rate, a, num_steps)
modelBK = BKTree(sigma, a, num_steps)
modelBDT = BDTTree(sigma, num_steps)

In [65]:
ec = europeanCallBondOption.value(settle_date, discount_curve, modelHW)
ac = americanCallBondOption.value(settle_date, discount_curve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   4.30650
American Call Value:   5.26581


In [66]:
ep = europeanPutBondOption.value(settle_date, discount_curve, modelHW)
ap = americanPutBondOption.value(settle_date, discount_curve, modelHW)
print("European Put Value: %9.5f" % ep)
print("American Put Value: %9.5f" % ap)

European Put Value:   0.00000
American Put Value:   0.00454


Copyright (c) Dominic O'Kane 2020